In [71]:
import argparse
import pandas as pd
import os
from pathlib import Path
import numpy as np
import h5py 


In [58]:
HOME_PATH = '/gpfs/data/denizlab/Users/bz1030/data/OAI_processed/'

In [67]:
summary_path = os.path.join(HOME_PATH,'summary.csv')
df = pd.read_csv(summary_path)
print('Total images {}'.format(df.shape[0]))

Total images 44865


In [62]:
hdf5_path = os.path.join(HOME_PATH,'mix')

In [66]:
count_hdf5 = 0
for each in os.listdir(hdf5_path):
    count_hdf5 += len(os.listdir(os.path.join(hdf5_path,each)))
count_hdf5

45952

In [44]:
filename ='/gpfs/data/denizlab/Users/bz1030/data/OAI_processed/mix/00m/9000099_00m_LEFT_KNEE.hdf5'
f = h5py.File(filename, 'r')

# List all groups
print("Keys: %s" % f.keys())
a_group_key = list(f.keys())

# Get the data


Keys: KeysView(<HDF5 file "9000099_00m_LEFT_KNEE.hdf5" (mode r)>)


In [73]:
img = read_hdf5(filename)

In [72]:
def read_hdf5(file_path):
    f = h5py.File(file_path, 'r')
    img = list(f['data'])
    return np.stack(img,axis=0)
    

In [78]:
df.head()

,File Name,Folder,Participant ID,Study Date,Bar Code,Description,Image Size,KLG,Method,IsSuccessful,Month
0,9000296_96m_LEFT_KNEE.hdf5,/gpfs/data/denizlab/Datasets/OAI_original/96m/...,9000296,20120720,16604011302,LEFT,1024x1024,3.0,mix,1,96m
1,9000296_96m_RIGHT_KNEE.hdf5,/gpfs/data/denizlab/Datasets/OAI_original/96m/...,9000296,20120720,16604011302,RIGHT,1024x1024,2.0,mix,1,96m
2,9000798_96m_LEFT_KNEE.hdf5,/gpfs/data/denizlab/Datasets/OAI_original/96m/...,9000798,20120928,16603896401,LEFT,1024x1024,4.0,mix,1,96m
3,9000798_96m_RIGHT_KNEE.hdf5,/gpfs/data/denizlab/Datasets/OAI_original/96m/...,9000798,20120928,16603896401,RIGHT,1024x1024,1.0,mix,1,96m
4,9001897_96m_LEFT_KNEE.hdf5,/gpfs/data/denizlab/Datasets/OAI_original/96m/...,9001897,20130116,16603998101,LEFT,1024x1024,0.0,mix,1,96m


In [100]:
HOME_PATH = '/gpfs/data/denizlab/Users/bz1030/data/OAI_processed/mix/'
def image_generator(summary, home_path,batch_size = 64):
    indexs = summary.index.tolist()
    while True:
        # Select files (paths/indices) for the batch
        batch_indexs = np.random.choice(indexs,size=batch_size)
        batch_input = []
        batch_output = [] 

        # Read in each input, perform preprocessing and get labels
        print(len(batch_indexs))
        for row in batch_indexs:
            row = summary.loc[row]
            file_name = row['File Name']
            month = row['Month']
            label = row['KLG']
            file_path = os.path.join(home_path,month,file_name)
            img = read_hdf5(file_path)
            batch_input.append(img)
            batch_output.append(label)
        # Return a tuple of (input,output) to feed the network

        batch_x = np.stack(batch_input,axis = 0)
        batch_y = np.stack(batch_output,axis = 0)
        yield( batch_x, batch_y )

In [101]:
a = image_generator(df,HOME_PATH)

In [102]:
x,y = next(a)

64


In [103]:
x.shape

(64, 1024, 1024)

In [104]:
y.shape

(64,)